In [ ]:
# Databricks notebook source
# COMMAND ----------
# MAGIC %md
# MAGIC # Data Processing with PySpark
# MAGIC This notebook demonstrates loading data from a flat file and SQL Server, performing transformations, and writing the results back to a SQL Server table.

# COMMAND ----------
# MAGIC
# Import necessary libraries
import logging
from pyspark.sql import functions as F

# Initialize logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# COMMAND ----------
# MAGIC
# Securely retrieve credentials using Databricks utilities
sql_server_user = dbutils.secrets.get(scope="my_scope", key="sql_server_user")
sql_server_password = dbutils.secrets.get(scope="my_scope", key="sql_server_password")

# COMMAND ----------
# MAGIC
# Load Flat File Data
def load_flat_file_data():
    try:
        logger.info("Loading flat file data into DataFrame.")
        df = spark.read.format("csv") \
            .option("header", "true") \
            .option("inferSchema", "true") \
            .load("dbfs:/FileStore/tables/SampleCurrencyData.txt")
        logger.info("Flat file data loaded successfully.")
        return df
    except Exception as e:
        logger.error(f"Error loading flat file data: {e}")
        raise

flat_file_df = load_flat_file_data()

# COMMAND ----------
# MAGIC
# Load SQL Server Data for Lookups
def load_sql_server_data():
    try:
        logger.info("Loading DimCurrency data from SQL Server.")
        dim_currency_df = spark.read.format("jdbc") \
            .option("url", "jdbc:sqlserver://localhost;databaseName=AdventureWorksDW2012") \
            .option("dbtable", "dbo.DimCurrency") \
            .option("user", sql_server_user) \
            .option("password", sql_server_password) \
            .load()
        logger.info("DimCurrency data loaded successfully.")

        logger.info("Loading DimDate data from SQL Server.")
        dim_date_df = spark.read.format("jdbc") \
            .option("url", "jdbc:sqlserver://localhost;databaseName=AdventureWorksDW2012") \
            .option("dbtable", "dbo.DimDate") \
            .option("user", sql_server_user) \
            .option("password", sql_server_password) \
            .load()
        logger.info("DimDate data loaded successfully.")
        
        return dim_currency_df, dim_date_df
    except Exception as e:
        logger.error(f"Error loading data from SQL Server: {e}")
        raise

dim_currency_df, dim_date_df = load_sql_server_data()

# COMMAND ----------
# MAGIC
# Perform Lookup Transformations
def perform_lookup_transformations(flat_file_df, dim_currency_df, dim_date_df):
    try:
        logger.info("Performing lookup transformations.")
        enriched_df = flat_file_df \
            .join(dim_currency_df, F.col('CurrencyID') == F.col('CurrencyAlternateKey'), "inner") \
            .join(dim_date_df, F.col('CurrencyDate') == F.col('FullDateAlternateKey'), "inner") \
            .select(
                F.col('CurrencyKey').alias('CurrencyKey'),
                F.col('DateKey').alias('DateKey'),
                F.col('AverageRate').alias('AverageRate'),
                F.col('EndOfDayRate').alias('EndOfDayRate')
            )
        logger.info("Lookup transformations completed successfully.")
        return enriched_df
    except Exception as e:
        logger.error(f"Error during lookup transformations: {e}")
        raise

enriched_df = perform_lookup_transformations(flat_file_df, dim_currency_df, dim_date_df)

# COMMAND ----------
# MAGIC
# Load Transformed Data into FactCurrency Table
def load_transformed_data(enriched_df):
    try:
        logger.info("Loading transformed data into FactCurrency table.")
        enriched_df.write.format("jdbc") \
            .option("url", "jdbc:sqlserver://localhost;databaseName=AdventureWorksDW2012") \
            .option("dbtable", "dbo.FactCurrency") \
            .option("user", sql_server_user) \
            .option("password", sql_server_password) \
            .mode("append") \
            .save()
        logger.info("Data loaded into FactCurrency table successfully.")
    except Exception as e:
        logger.error(f"Error loading data into FactCurrency table: {e}")
        raise

load_transformed_data(enriched_df)
